## Ceneo Scraper

## Biblioteki

In [ ]:
import requests
from bs4 import BeautifulSoup

## Wysyłanie żądania dostępu do zasobu do serwera

In [ ]:
product_id = "114700014"
url = f"https://www.ceneo.pl/{product_id}/opinie-2"
response = requests.get(url)
response.status_code

In [ ]:
page_dom = BeautifulSoup(response.text, "html.parser")
print(type(page_dom))
opinions = page_dom.select("div.js_product-review")
print(type(opinions))
opinion = opinions.pop(0)
print(type(opinion))
print(opinion)

## Ekstrakcja składowych pojedyńczej tabeli

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post_author-recomendation|recommendations|
|gwiazdki|span.user-post_score_count|rating|
|treść|div.user-post__text|content|
|lista zalet|div.review-future__title--positives|pros|
|lista wad|div.review-future__title--negatives|cons|
|dla ilu przydatna|span[id^="votes-yes"]|useful|
|dla ilu nieprzydatna|span[id^="votes-no"]|useless|
|data wystawienia|span.user-post_published > time:nth-child(1)["datetime"]|post_date|
|data zakupu|span.user-post_published > time:nth-child(0)["datetime"]|purchase_date|

In [ ]:
opinion_id = opinion[["data-entry-id"]]
author = opinion.select_one("")
#